In [ ]:
from pynq import Overlay
from pynq import GPIO
import time
import hashlib

In [ ]:
overlay = Overlay("/home/xilinx/cartridge.bit")
c = None

In [ ]:
gpio_link_clock = GPIO(GPIO.get_gpio_pin(38), "out")
gpio_link_dir_clock = GPIO(GPIO.get_gpio_pin(39), "out")
gpio_link_data = GPIO(GPIO.get_gpio_pin(40), "out")
gpio_link_dir_data = GPIO(GPIO.get_gpio_pin(41), "out")
gpio_link_in = GPIO(GPIO.get_gpio_pin(42), "out")
gpio_link_dir_in = GPIO(GPIO.get_gpio_pin(43), "out")
gpio_link_out = GPIO(GPIO.get_gpio_pin(44), "out")
gpio_link_dir_out = GPIO(GPIO.get_gpio_pin(45), "out")

In [ ]:
gpio_link_dir_in.write(0)
gpio_link_dir_data.write(0)
gpio_link_dir_out.write(0)
gpio_link_dir_clock.write(0)

In [ ]:
class Cartridge:
    def __init__(self):
        self.gpio_wr = GPIO(GPIO.get_gpio_pin(1), "out")
        self.gpio_rd = GPIO(GPIO.get_gpio_pin(2), "out")
        self.gpio_cs = GPIO(GPIO.get_gpio_pin(3), "out")
        self.gpio_cs2 = GPIO(GPIO.get_gpio_pin(4), "out")
        self.gpio_phi = GPIO(GPIO.get_gpio_pin(5), "out")
        self.gpio_irq = GPIO(GPIO.get_gpio_pin(6), "in")
        
        self.gpio_oe = GPIO(GPIO.get_gpio_pin(7), "out")
        self.gpio_dir_A_hi = GPIO(GPIO.get_gpio_pin(8), "out")
        self.gpio_dir_A_lo = GPIO(GPIO.get_gpio_pin(9), "out")
        self.gpio_dir_ctrl = GPIO(GPIO.get_gpio_pin(10), "out")
        self.gpio_dir_D = GPIO(GPIO.get_gpio_pin(11), "out")
        self.gpio_dir_cs2 = GPIO(GPIO.get_gpio_pin(12), "out")
        self.gpio_dir_irq = GPIO(GPIO.get_gpio_pin(13), "out")

        self.gpio_ad = [GPIO(GPIO.get_gpio_pin(14 + i), "out") for i in range(24)]
        self.read_mode = True
        
        # Set initial directions
        self.gpio_dir_A_hi.write(1) # Output
        self.gpio_dir_A_lo.write(1) # Output
        self.gpio_dir_ctrl.write(1) # Output
        self.gpio_dir_D.write(1) # Output
        self.gpio_dir_cs2.write(1) # Output
        self.gpio_dir_irq.write(0) # Input
        self.gpio_oe.write(0) # Enable
        
        self.gpio_wr.write(1)
        self.gpio_rd.write(1)
        self.gpio_cs.write(1)
        self.gpio_cs2.write(1)
        self.gpio_phi.write(0)
        
        self._sleep()
        
    def _sleep(self):
        sleep_time = 2.0 / (16 * 1024 * 1024)
        end = time.monotonic() + sleep_time
        while time.monotonic() < end:
            pass
        
    def read_rom_nonsequential(self, address):
        # Set up all output for writing new address
        for g in self.gpio_ad:
            g.release()
        self.gpio_ad = [GPIO(GPIO.get_gpio_pin(14 + i), "out") for i in range(24)]
        self.gpio_dir_A_hi.write(1)
        self.gpio_dir_A_lo.write(1)
        self.gpio_dir_D.write(1)
        self.read_mode = False
        
        # Write full address
        for i in range(24):
            self.gpio_ad[i].write(address & 1)
            address = address >> 1
            
        # CS 1 -> 0 latches address
        self.gpio_cs.write(1)
        self._sleep()
        self.gpio_cs.write(0)
        
        data = self.read_rom_sequential()
        
        # CS should be left LOW when reading in this burst
        
        return data
        
    def read_rom_sequential(self):
        # Set up data bus for input
        if not self.read_mode:
            for g in self.gpio_ad[:16]:
                g.release()
            self.gpio_ad[:16] = [GPIO(GPIO.get_gpio_pin(14 + i), "in") for i in range(16)]
            self.gpio_dir_A_hi.write(0)
            self.gpio_dir_A_lo.write(0)
            self.read_mode = True
        
        # Pull read low.
        self.gpio_rd.write(0)
        self._sleep()
        
        # Read data
        data = 0
        for i in range(16):
            data |= self.gpio_ad[i].read() << i
            
        # Put read high.
        self.gpio_rd.write(1)
        
        return [data & 0xFF, (data >> 8) & 0xFF]
        
    def release(self):
        gpios = [self.gpio_wr, self.gpio_rd, self.gpio_cs, self.gpio_cs2, self.gpio_phi, self.gpio_irq] + self.gpio_ad
        for gpio in gpios:
            gpio.release()
        
if c is not None:
    c.release()
c = Cartridge()

In [ ]:
data = c.read_rom_nonsequential(3)
print([hex(x) for x in data])

In [ ]:
data = c.read_rom_sequential()
print([hex(x) for x in data])

In [ ]:
start_time = time.time()
rom_dump = c.read_rom_nonsequential(0)
while len(rom_dump) < 256:
    rom_dump.extend(c.read_rom_sequential())
end_time = time.time()
print(end_time - start_time)